# Notebook 9 - U(1) Clock Playground

**Level:** Intermediate  
**Topics:** U(1) clock model, THRML, lattice sampling, phase transitions, S1 geometry

> **Provenance:** This notebook is based on:
> - `light_theory_realm/models/README.md` (U1ClockTHRMLModel)
> - U(1) clock or THRML examples in the repo


In [ ]:
import numpy as np
import jax
import jax.numpy as jnp
import matplotlib.pyplot as plt

from light_theory_realm.models.classical_clock import U1ClockTHRMLModel

plt.rcParams["figure.figsize"] = (5, 5)


## 1. 8x8 U(1) clock lattice with q = 8


In [ ]:
L = 4   # lattice size -> 4x4 to keep sampling light
q = 8   # number of clock states (angles)

model = U1ClockTHRMLModel(lattice_size=L, q=q)

print(model)
print(f"Total spins: {model.total_spins}")


## 2. Sampling configurations at different temperatures

We will draw samples at three temperatures:

- Low temperature (ordered phase)
- Intermediate temperature
- High temperature (disordered phase)


In [ ]:

temperatures = [0.5, 1.0, 2.0]
configs = {}
key = jax.random.PRNGKey(0)

for T in temperatures:
    key, sub = jax.random.split(key)
    try:
        binary_samples = model.sample_states(T=T, n_warmup=10, n_samples=1, steps_per_sample=50, key=sub)
        phases = model.decode_phases(binary_samples)[0]
    except Exception as exc:
        # Fallback: simple random phases if THRML backend objects are unavailable
        print(f"Using random phase fallback at T={T}: {exc}")
        rand_states = jax.random.randint(sub, shape=(L * L,), minval=0, maxval=q)
        phases = (2 * jnp.pi * rand_states / q).reshape(L, L)
    configs[T] = phases.reshape(L, L)


> Note: If you have THRML fully installed and wired, you can remove the fallback and run the native `sample_states` path for exact clock sampling.


## 3. Visualizing phase configurations

We map each clock state k to an angle: theta_k = 2*pi*k/q, then display that angle as an image.


In [ ]:
def plot_phase_config(cfg, title=""):
    plt.figure()
    plt.imshow(cfg, origin="lower")
    plt.colorbar(label="Angle [rad]")
    plt.title(title)
    plt.xticks([])
    plt.yticks([])
    plt.tight_layout()
    plt.show()

for T, cfg in configs.items():
    plot_phase_config(cfg, title=f"U(1) clock configuration at T = {T}")


## 4. Geometry on S1 x ... x S1

Each lattice site carries a phase on the circle S1. The full state space is (S1)^(L^2), and the THRML dynamics explore this high-dimensional torus.

At low temperature, phases tend to align (ordered, small fluctuations). At high temperature, phases are more uniformly spread (disordered).

Next steps:

- Track an order parameter, such as the vector sum of phases.
- Plot the order parameter versus temperature.
- Relate this to phase transitions and geometry of the configuration manifold.
